In [33]:
import random

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from algorithm import ewm

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
palette = 'deep'
sns.set(font='SimHei', font_scale=1.2, palette=palette)  # 解决Seaborn中文显示问题

# 数据预处理

In [34]:
df = pd.read_excel('赋分后df.xlsx')
df

,1、您的性别(1-22题为单选题),2、您的专业,3、您所在的年级,4、您的性格,5、您最常通过哪种方式上网？,6、您每周的上网时长大约是多少？,7、您是否使用过学习软件工具？,8、您是在什么时间使用学习软件工具呢？,9、您会将自己的资料传到网上和别人进行分享吗？,10、您是否想获取全国各高校的学习资源？,...,21、您觉得人工智能工具在未来是否可以取代教师？,22、当人工智能工具与教育结合到一定程度时，您觉得学生该如何去适应？,23、在网络中您主要进行哪些活动？,24、您认为学习软件与课堂教学相比较最大的优势是什么？,25、进入大学后在学习中困扰您的问题是什么？,26、对于利用学习软件进行学习这种形式，您最关注的是,27、您考虑过使用人工智能工具的哪些安全性？,28、您认为以下哪些方面对人工智能学习工具很重要？,29、您心目中的人工智能学习工具应该具有以下哪些功能？,30、您认为人工智能学习工具融合到哪个学习环节？
0,1,1,2,6,2,2,2,4,5,2,...,4,4,1,1,1,1,1,1,1,1
1,0,2,2,1,2,1,1,1,5,2,...,1,4,1,3,4,4,6,3,5,4
2,1,1,2,3,2,2,2,4,2,2,...,2,4,7,3,4,4,6,3,5,3
3,0,2,2,4,4,4,2,4,5,2,...,4,4,7,3,4,4,6,3,5,4
4,1,2,1,3,2,2,2,4,2,2,...,3,4,5,2,2,3,3,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600,0,2,1,1,1,5,2,1,2,2,...,2,4,6,3,3,3,6,2,4,1
4601,0,2,1,2,2,3,2,4,4,2,...,2,4,6,3,2,4,6,3,5,4
4602,0,2,1,2,1,2,2,4,4,2,...,2,4,7,3,1,4,6,3,4,4
4603,0,2,1,1,2,3,2,1,3,2,...,4,4,4,3,3,4,6,3,5,4


In [35]:
指标权重dict = {
    '对AI的需求程度(Q1)': {
        '13、您是否有通过人工智能学习工具帮助完成作业的想法？': 1,
        '14、您是否有通过人工智能学习工具帮您完成小测验的想法？': 1,
        '15、您是否有通过人工智能学习工具帮助您完成论文的想法？': 1,
        '17、对于大学生使用人工智能学习工具您是否赞同？': 1,
        '19、您对人工智能学习工具回答问题的可信度持何种态度？': 1,
        '28、您认为以下哪些方面对人工智能学习工具很重要？': 1,
    },
    '对网络认识的深度(Q2)': {
        '6、您每周的上网时长大约是多少？': 1,
        '8、您是在什么时间使用学习软件工具呢？': 1,
        '9、您会将自己的资料传到网上和别人进行分享吗？': 1,
        '23、在网络中您主要进行哪些活动？': 1,
    },
    '倾向于使用软件进行的程度(Q3)': {
        '7、您是否使用过学习软件工具？': 1,
        '8、您是在什么时间使用学习软件工具呢？': 1,
        '11、您的老师是否推荐过同学们使用过某种学习软件？': 1,
        '19、您对人工智能学习工具回答问题的可信度持何种态度？': 1,
        '26、对于利用学习软件进行学习这种形式，您最关注的是': 1,
        '30、您认为人工智能学习工具融合到哪个学习环节？': 1,
    },
    '对学习的热爱程度(Q4)': {
        '9、您会将自己的资料传到网上和别人进行分享吗？': 1,
        '10、您是否想获取全国各高校的学习资源？': 1,
        '12、若有人工智能学习工具，您是否会选择使用？': 1,
        '20、如果您使用人工智能学习工具，更希望得到什么的效果？': 1,
        '25、进入大学后在学习中困扰您的问题是什么？': 1,
    },
    '对AI的认可程度(Q5)': {
        '21、您觉得人工智能工具在未来是否可以取代教师？': 1,
        '22、当人工智能工具与教育结合到一定程度时，您觉得学生该如何去适应？': 1,
        '24、您认为学习软件与课堂教学相比较最大的优势是什么？': 1,
        '27、您考虑过使用人工智能工具的哪些安全性？': 1,
        '30、您认为人工智能学习工具融合到哪个学习环节？': 1,
    },
    # '对AI工具否定程度(Q6)': {
    #     '16、若有人工智能学习工具，影响您使用人工智能学习软件的原因是什么？': 1,
    #     '18、若您使用人工智能学习工具最不希望出现以下哪些事？': 1,
    # },
}

In [36]:
def calcEwm(一级指标):
    二级指标 = list(指标权重dict[一级指标].keys())
    currentDf = df.loc[:, 二级指标].copy()
    _, _, weight = ewm.Entropy_Weight_Method(currentDf.values, [1] * len(currentDf.columns), scale_min=0.00001,
                                             scale_max=0.99999, display=True, indexCol=二级指标).fit()
    指标权重dict[一级指标] = dict(zip(二级指标, weight))


calcEwm('对AI的需求程度(Q1)')

┌────────────────────────────────────────────────────────┬─────────────────────┐
│ index                                                  │ Index weight        │
├────────────────────────────────────────────────────────┼─────────────────────┤
│ 13、您是否有通过人工智能学习工具帮助完成作业的想法？   │ 0.19105317366172253 │
│ 14、您是否有通过人工智能学习工具帮您完成小测验的想法？ │ 0.19664015955938982 │
│ 15、您是否有通过人工智能学习工具帮助您完成论文的想法？ │ 0.19938279938254982 │
│ 17、对于大学生使用人工智能学习工具您是否赞同？         │ 0.17631193373822923 │
│ 19、您对人工智能学习工具回答问题的可信度持何种态度？   │ 0.06708662266985567 │
│ 28、您认为以下哪些方面对人工智能学习工具很重要？       │ 0.1695253109882529  │
└────────────────────────────────────────────────────────┴─────────────────────┘


In [37]:
for i in 指标权重dict:
    print(f'--------{i}-------------')
    calcEwm(i)

--------对AI的需求程度(Q1)-------------
┌────────────────────────────────────────────────────────┬─────────────────────┐
│ index                                                  │ Index weight        │
├────────────────────────────────────────────────────────┼─────────────────────┤
│ 13、您是否有通过人工智能学习工具帮助完成作业的想法？   │ 0.19105317366172253 │
│ 14、您是否有通过人工智能学习工具帮您完成小测验的想法？ │ 0.19664015955938982 │
│ 15、您是否有通过人工智能学习工具帮助您完成论文的想法？ │ 0.19938279938254982 │
│ 17、对于大学生使用人工智能学习工具您是否赞同？         │ 0.17631193373822923 │
│ 19、您对人工智能学习工具回答问题的可信度持何种态度？   │ 0.06708662266985567 │
│ 28、您认为以下哪些方面对人工智能学习工具很重要？       │ 0.1695253109882529  │
└────────────────────────────────────────────────────────┴─────────────────────┘
--------对网络认识的深度(Q2)-------------
┌───────────────────────────────────────────────┬─────────────────────┐
│ index                                         │ Index weight        │
├───────────────────────────────────────────────┼─────────────────────┤
│ 6、您每周的上网时长大约是多少？               │ 0.1403419234605448

In [38]:
一级指标df = df.iloc[:, :4].copy()
一级指标df

,1、您的性别(1-22题为单选题),2、您的专业,3、您所在的年级,4、您的性格
0,1,1,2,6
1,0,2,2,1
2,1,1,2,3
3,0,2,2,4
4,1,2,1,3
...,...,...,...,...
4600,0,2,1,1
4601,0,2,1,2
4602,0,2,1,2
4603,0,2,1,1


In [39]:
for k1, v1 in 指标权重dict.items():
    sum = np.zeros(len(一级指标df))
    for k2, v2 in v1.items():
        sum += v2 * df[k2]

    print(f'-----{k1}---------')
    print(sum)
    一级指标df[k1] = sum

一级指标df

-----对AI的需求程度(Q1)---------
0       2.660949
1       1.339051
2       2.932913
3       1.758761
4       2.763388
          ...   
4600    2.763388
4601    3.000000
4602    2.736273
4603    2.932913
4604    1.758761
Name: 13、您是否有通过人工智能学习工具帮助完成作业的想法？, Length: 4605, dtype: float64
-----对网络认识的深度(Q2)---------
0       3.242675
1       2.150800
2       3.908258
3       5.052041
4       3.398697
          ...   
4600    3.122970
4601    4.369219
4602    4.483658
4603    2.620425
4604    1.963506
Name: 6、您每周的上网时长大约是多少？, Length: 4605, dtype: float64
-----倾向于使用软件进行的程度(Q3)---------
0       1.923979
1       2.704863
2       3.021067
3       3.314578
4       1.999760
          ...   
4600    1.493494
4601    3.389580
4602    3.314578
4603    2.569050
4604    1.927778
Name: 7、您是否使用过学习软件工具？, Length: 4605, dtype: float64
-----对学习的热爱程度(Q4)---------
0       2.484637
1       2.962907
2       2.933170
3       3.608576
4       1.852059
          ...   
4600    2.558524
4601    2.634148
4602    2.259502
4603 

,1、您的性别(1-22题为单选题),2、您的专业,3、您所在的年级,4、您的性格,对AI的需求程度(Q1),对网络认识的深度(Q2),倾向于使用软件进行的程度(Q3),对学习的热爱程度(Q4),对AI的认可程度(Q5)
0,1,1,2,6,2.660949,3.242675,1.923979,2.484637,1.474721
1,0,2,2,1,1.339051,2.150800,2.704863,2.962907,3.670108
2,1,1,2,3,2.932913,3.908258,3.021067,2.933170,3.485175
3,0,2,2,4,1.758761,5.052041,3.314578,3.608576,4.069555
4,1,2,1,3,2.763388,3.398697,1.999760,1.852059,2.062994
...,...,...,...,...,...,...,...,...,...
4600,0,2,1,1,2.763388,3.122970,1.493494,2.558524,2.849011
4601,0,2,1,2,3.000000,4.369219,3.389580,2.634148,3.803257
4602,0,2,1,2,2.736273,4.483658,3.314578,2.259502,3.803257
4603,0,2,1,1,2.932913,2.620425,2.569050,2.783659,4.069555


In [40]:
一级指标df.to_excel('一级指标结果.xlsx', index=False)

In [41]:
matrix = 一级指标df.iloc[:, 4:]
matrix

,对AI的需求程度(Q1),对网络认识的深度(Q2),倾向于使用软件进行的程度(Q3),对学习的热爱程度(Q4),对AI的认可程度(Q5)
0,2.660949,3.242675,1.923979,2.484637,1.474721
1,1.339051,2.150800,2.704863,2.962907,3.670108
2,2.932913,3.908258,3.021067,2.933170,3.485175
3,1.758761,5.052041,3.314578,3.608576,4.069555
4,2.763388,3.398697,1.999760,1.852059,2.062994
...,...,...,...,...,...
4600,2.763388,3.122970,1.493494,2.558524,2.849011
4601,3.000000,4.369219,3.389580,2.634148,3.803257
4602,2.736273,4.483658,3.314578,2.259502,3.803257
4603,2.932913,2.620425,2.569050,2.783659,4.069555


In [42]:
_, _, weights = ewm.Entropy_Weight_Method(matrix.values, [1] * matrix.shape[1], scale_min=0.00001,
                                          scale_max=0.99999, display=True, indexCol=matrix.columns).fit()

┌──────────────────────────────┬─────────────────────┐
│ index                        │ Index weight        │
├──────────────────────────────┼─────────────────────┤
│ 对AI的需求程度(Q1)           │ 0.23691834002774398 │
│ 对网络认识的深度(Q2)         │ 0.1265049169251752  │
│ 倾向于使用软件进行的程度(Q3) │ 0.19320241159128132 │
│ 对学习的热爱程度(Q4)         │ 0.14435884829203316 │
│ 对AI的认可程度(Q5)           │ 0.29901548316376636 │
└──────────────────────────────┴─────────────────────┘


In [43]:
from algorithm.topsis import topsis
# pd.DataFrame().rank()
result, Z, weight = topsis(matrix, weights)
result

,对AI的需求程度(Q1),对网络认识的深度(Q2),倾向于使用软件进行的程度(Q3),对学习的热爱程度(Q4),对AI的认可程度(Q5),正理想解,负理想解,综合得分指数,排序
0,0.017864,0.013536,0.012230,0.015204,0.008306,0.009970,0.007985,0.444707,3042.0
1,0.008990,0.008978,0.017193,0.018131,0.020671,0.007800,0.010888,0.582629,1639.0
2,0.019690,0.016314,0.019203,0.017949,0.019630,0.003469,0.013131,0.791026,252.0
3,0.011807,0.021088,0.021069,0.022082,0.022921,0.004104,0.014650,0.781177,284.0
4,0.018552,0.014187,0.012711,0.011333,0.011620,0.008933,0.008355,0.483300,2622.0
...,...,...,...,...,...,...,...,...,...
4600,0.018552,0.013036,0.009493,0.015656,0.016047,0.007788,0.009655,0.553493,1939.0
4601,0.020141,0.018238,0.021545,0.016119,0.021421,0.002893,0.014283,0.831595,139.0
4602,0.018370,0.018716,0.021069,0.013827,0.021421,0.003653,0.013639,0.788744,259.0
4603,0.019690,0.010938,0.016330,0.017034,0.022921,0.005157,0.013208,0.719192,566.0


In [50]:
weight

array([0.23691834, 0.12650492, 0.19320241, 0.14435885, 0.29901548])

In [49]:
Z

,对AI的需求程度(Q1),对网络认识的深度(Q2),倾向于使用软件进行的程度(Q3),对学习的热爱程度(Q4),对AI的认可程度(Q5)
负理想解,0.006714,0.004174,0.006356,0.005214,0.005632
正理想解,0.020141,0.022737,0.021545,0.022082,0.022921


In [44]:
result_width = 一级指标df.join(result.iloc[:, -4:])
result_width

,1、您的性别(1-22题为单选题),2、您的专业,3、您所在的年级,4、您的性格,对AI的需求程度(Q1),对网络认识的深度(Q2),倾向于使用软件进行的程度(Q3),对学习的热爱程度(Q4),对AI的认可程度(Q5),正理想解,负理想解,综合得分指数,排序
0,1,1,2,6,2.660949,3.242675,1.923979,2.484637,1.474721,0.009970,0.007985,0.444707,3042.0
1,0,2,2,1,1.339051,2.150800,2.704863,2.962907,3.670108,0.007800,0.010888,0.582629,1639.0
2,1,1,2,3,2.932913,3.908258,3.021067,2.933170,3.485175,0.003469,0.013131,0.791026,252.0
3,0,2,2,4,1.758761,5.052041,3.314578,3.608576,4.069555,0.004104,0.014650,0.781177,284.0
4,1,2,1,3,2.763388,3.398697,1.999760,1.852059,2.062994,0.008933,0.008355,0.483300,2622.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600,0,2,1,1,2.763388,3.122970,1.493494,2.558524,2.849011,0.007788,0.009655,0.553493,1939.0
4601,0,2,1,2,3.000000,4.369219,3.389580,2.634148,3.803257,0.002893,0.014283,0.831595,139.0
4602,0,2,1,2,2.736273,4.483658,3.314578,2.259502,3.803257,0.003653,0.013639,0.788744,259.0
4603,0,2,1,1,2.932913,2.620425,2.569050,2.783659,4.069555,0.005157,0.013208,0.719192,566.0


In [45]:
result_width.to_excel('一级指标topsis结果.xlsx', index=False)

# 汇总表

In [46]:
originDf = pd.read_excel('附件2：调查数据.xlsx')
originDf

,序号,1、您的性别(1-22题为单选题),2、您的专业,3、您所在的年级,4、您的性格,5、您最常通过哪种方式上网？,6、您每周的上网时长大约是多少？,7、您是否使用过学习软件工具？,8、您是在什么时间使用学习软件工具呢？,9、您会将自己的资料传到网上和别人进行分享吗？,...,21、您觉得人工智能工具在未来是否可以取代教师？,22、当人工智能工具与教育结合到一定程度时，您觉得学生该如何去适应？,23、在网络中您主要进行哪些活动？,24、您认为学习软件与课堂教学相比较最大的优势是什么？,25、进入大学后在学习中困扰您的问题是什么？,26、对于利用学习软件进行学习这种形式，您最关注的是,27、您考虑过使用人工智能工具的哪些安全性？,28、您认为以下哪些方面对人工智能学习工具很重要？,29、您心目中的人工智能学习工具应该具有以下哪些功能？,30、您认为人工智能学习工具融合到哪个学习环节？
0,1,女,文史类,大二,其他,用手机上网,7小时以下,是,平时有时间就使用,完全会,...,非常可能,积极利用新的学习方式和工具,学习、查资料,真题全面,学习的相关经验缺乏,学习效果,软件安全级别,知识来源的资格审核,性能优越,教师传授
1,2,男,理工类,大二,安静型,用手机上网,不上网,否,说不清,完全会,...,不清楚,积极利用新的学习方式和工具,其他,真题全面┋可以重复学习┋资料全面,学习的相关经验缺乏┋专业疑难问题得不到解决┋学习时间安排不充裕┋不会正确的学习方法,学习效果┋学习资源┋操作方便┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋知识库的更新频率┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋稳定┋不收费,教师传授┋课后消化┋评价反馈┋其他
2,3,女,文史类,大二,温顺型,用手机上网,7小时以下,是,平时有时间就使用,很少会,...,不可能,积极利用新的学习方式和工具,学习、查资料┋浏览新闻┋收发邮件┋娱乐游戏┋聊天交友┋资源下载┋上网购物,真题全面┋可以重复学习┋资料全面,学习的相关经验缺乏┋专业疑难问题得不到解决┋学习时间安排不充裕┋不会正确的学习方法,学习效果┋学习资源┋操作方便┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋知识库的更新频率┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋稳定┋不收费,教师传授┋课后消化┋评价反馈
3,4,男,理工类,大二,坚定型,在网吧上,14-20小时,是,平时有时间就使用,完全会,...,非常可能,积极利用新的学习方式和工具,学习、查资料┋浏览新闻┋收发邮件┋娱乐游戏┋聊天交友┋资源下载┋上网购物,真题全面┋可以重复学习┋资料全面,学习的相关经验缺乏┋专业疑难问题得不到解决┋学习时间安排不充裕┋不会正确的学习方法,学习效果┋学习资源┋操作方便┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋知识库的更新频率┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋稳定┋不收费,教师传授┋课后消化┋评价反馈┋其他
4,5,女,理工类,大一,温顺型,用手机上网,7小时以下,是,平时有时间就使用,很少会,...,有可能,积极利用新的学习方式和工具,学习、查资料┋浏览新闻┋娱乐游戏┋聊天交友┋上网购物,真题全面┋可以重复学习,专业疑难问题得不到解决┋学习时间安排不充裕,学习效果┋学习资源┋操作方便,软件安全级别┋个人信息安全┋运行安全,知识来源的资格审核┋知识库的更新频率,性能优越┋运行速度快┋稳定,教师传授
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600,4601,男,理工类,大一,安静型,在寝室用笔记本上网,20小时以上,是,说不清,很少会,...,不可能,积极利用新的学习方式和工具,学习、查资料┋浏览新闻┋娱乐游戏┋聊天交友┋资源下载┋上网购物,真题全面┋可以重复学习┋资料全面,学习的相关经验缺乏┋专业疑难问题得不到解决┋学习时间安排不充裕,学习效果┋学习资源┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋不收费,课后消化
4601,4602,男,理工类,大一,外向型,用手机上网,7-14小时,是,平时有时间就使用,大多数时候会,...,不可能,积极利用新的学习方式和工具,学习、查资料┋娱乐游戏┋聊天交友┋资源下载┋上网购物┋其他,真题全面┋可以重复学习┋资料全面,学习的相关经验缺乏┋不会正确的学习方法,学习效果┋学习资源┋操作方便┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋知识库的更新频率┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋稳定┋不收费,教师传授┋课后消化┋评价反馈┋其他
4602,4603,男,理工类,大一,外向型,在寝室用笔记本上网,7小时以下,是,平时有时间就使用,大多数时候会,...,不可能,积极利用新的学习方式和工具,学习、查资料┋浏览新闻┋收发邮件┋娱乐游戏┋聊天交友┋资源下载┋上网购物,真题全面┋可以重复学习┋资料全面,不会正确的学习方法,学习效果┋学习资源┋操作方便┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋知识库的更新频率┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋稳定,教师传授┋课后消化┋评价反馈┋其他
4603,4604,男,理工类,大一,安静型,用手机上网,7-14小时,是,说不清,有时会,...,非常可能,积极利用新的学习方式和工具,浏览新闻┋娱乐游戏┋聊天交友┋上网购物,真题全面┋可以重复学习┋资料全面,专业疑难问题得不到解决┋学习时间安排不充裕┋不会正确的学习方法,学习效果┋学习资源┋操作方便┋学习费用,软件安全级别┋网络安全能力┋个人信息安全┋数据安全┋运行安全┋服务器安全,知识来源的资格审核┋知识库的更新频率┋是否有定期的审核,性能优越┋知识面广┋运行速度快┋稳定┋不收费,教师传授┋课后消化┋评价反馈┋其他


In [47]:
汇总df = originDf.copy()
汇总df = 汇总df.join(result)
汇总df

,序号,1、您的性别(1-22题为单选题),2、您的专业,3、您所在的年级,4、您的性格,5、您最常通过哪种方式上网？,6、您每周的上网时长大约是多少？,7、您是否使用过学习软件工具？,8、您是在什么时间使用学习软件工具呢？,9、您会将自己的资料传到网上和别人进行分享吗？,...,30、您认为人工智能学习工具融合到哪个学习环节？,对AI的需求程度(Q1),对网络认识的深度(Q2),倾向于使用软件进行的程度(Q3),对学习的热爱程度(Q4),对AI的认可程度(Q5),正理想解,负理想解,综合得分指数,排序
0,1,女,文史类,大二,其他,用手机上网,7小时以下,是,平时有时间就使用,完全会,...,教师传授,0.017864,0.013536,0.012230,0.015204,0.008306,0.009970,0.007985,0.444707,3042.0
1,2,男,理工类,大二,安静型,用手机上网,不上网,否,说不清,完全会,...,教师传授┋课后消化┋评价反馈┋其他,0.008990,0.008978,0.017193,0.018131,0.020671,0.007800,0.010888,0.582629,1639.0
2,3,女,文史类,大二,温顺型,用手机上网,7小时以下,是,平时有时间就使用,很少会,...,教师传授┋课后消化┋评价反馈,0.019690,0.016314,0.019203,0.017949,0.019630,0.003469,0.013131,0.791026,252.0
3,4,男,理工类,大二,坚定型,在网吧上,14-20小时,是,平时有时间就使用,完全会,...,教师传授┋课后消化┋评价反馈┋其他,0.011807,0.021088,0.021069,0.022082,0.022921,0.004104,0.014650,0.781177,284.0
4,5,女,理工类,大一,温顺型,用手机上网,7小时以下,是,平时有时间就使用,很少会,...,教师传授,0.018552,0.014187,0.012711,0.011333,0.011620,0.008933,0.008355,0.483300,2622.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600,4601,男,理工类,大一,安静型,在寝室用笔记本上网,20小时以上,是,说不清,很少会,...,课后消化,0.018552,0.013036,0.009493,0.015656,0.016047,0.007788,0.009655,0.553493,1939.0
4601,4602,男,理工类,大一,外向型,用手机上网,7-14小时,是,平时有时间就使用,大多数时候会,...,教师传授┋课后消化┋评价反馈┋其他,0.020141,0.018238,0.021545,0.016119,0.021421,0.002893,0.014283,0.831595,139.0
4602,4603,男,理工类,大一,外向型,在寝室用笔记本上网,7小时以下,是,平时有时间就使用,大多数时候会,...,教师传授┋课后消化┋评价反馈┋其他,0.018370,0.018716,0.021069,0.013827,0.021421,0.003653,0.013639,0.788744,259.0
4603,4604,男,理工类,大一,安静型,用手机上网,7-14小时,是,说不清,有时会,...,教师传授┋课后消化┋评价反馈┋其他,0.019690,0.010938,0.016330,0.017034,0.022921,0.005157,0.013208,0.719192,566.0


In [48]:
汇总df.to_excel('topsis和原始问卷汇总结果.xlsx', index=False)